# Linear algebra Homework assignment 3

In [66]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

## Problem 1 - 4

In [67]:
# [TODO]
def compute_lookat(azim: float, elev: float):
    """
        Compute the look at vector. (Definition in Figure 3)
        azim: float, degree in [-180, 180],
        elev: float, degree in [-180, 180]
    """
    # For the definition of azim and elev, check
    # https://matplotlib.org/stable/api/toolkits/mplot3d/view_angles.html
    azim_rad= np.radians(azim)
    elev_rad= np.radians(elev)
    a = np.cos(np.radians(90-elev))* np.sin(azim_rad)
    b = np.cos(np.radians(90-elev))* np.cos(azim_rad)
    c = np.sin(np.radians(90-elev))
    lookat = np.array([a,b,c])
    magnitude=np.linalg.norm(lookat)
    if magnitude != 0:
      lookat=lookat/magnitude
    return lookat

In [68]:
# [TODO]
def compute_normal(P1: tuple, P2: tuple, P3: tuple):
    """
        Compute the normal vector, given P1, P2, P3 in counter-clockwise order.
    """
    P1_list= np.array(P1)
    P2_list= np.array(P2)
    P3_list= np.array(P3)
    # Compute vectors representing edges of the triangle
    v1 = P2-P1 # distance p2 to p1
    v2 = P3-P1 #distance p3 to p1

    # Compute the cross product of the edges to get the normal vector
    normal = np.cross(v1,v2)

    return normal

In [69]:
# [TODO]
def visible(face_normal: np.ndarray, lookat: np.ndarray):
    """
        Given a normal vector of a face, determine if the face (outward-facing side) is visible
    """
    lookat= lookat/np.linalg.norm(lookat)
    face_normal= face_normal/ np.linalg.norm(face_normal)  #normalize
    dotProduct= np.dot(face_normal, lookat) #dot operation
    # use the normal vector of the triangle and the lookat direction
    # to determine the visibility.

    return dotProduct > 0

In [70]:
# [TODO] compute cos(β) in Figure3
def compute_intensity(face_normal: np.ndarray, lookat: np.ndarray, lightsource: np.ndarray):
    """
        Given normal vector of a face (face_normal), viewing vector (lookat) and lightsource (lightsource),
        compute the specular intensity.
    """
    face_normal = face_normal / np.linalg.norm(face_normal)
    lightsource = lightsource / np.linalg.norm(lightsource)
    lookat = lookat / np.linalg.norm(lookat)
    dot_product_ln = np.dot(face_normal, lightsource)
    reflect = 2 * dot_product_ln * face_normal - lightsource
    intensity = np.dot(reflect, lookat)
    return intensity

## Problem 5
Perform at least two tasks from the following list, and describe what you have done

- Change a model (please choose a convex object).
- Give each face of the model a different color.
- Change the movemnt of the light source.
- Other interesting changes

Description:

I modified the model so that each face of the model has a different color. I did this by by using a colormap (hsv). This ensures each side will have a different color.

I also changed the movement of the light source by rotating the light source around the object in each frame. The new position of the light source is calculated based on trigonometric functions (cos and sin) and scaled by light_radius. The effect of the moving light source is reflected in the shading of the object faces.

I also changed the object to a tetrahedron which has 4 vertices, 6 edges, and 4 faces. In the png file only 3 faces are seen because it depends on the perspective view.

I decided to put all the function into the main function because in my opinion, it is easier to navigate and change the code when i can see everthing in the same place.

There is 2 functions because one of them is used to generate the object before the change in the light source movement while the other is used to generate the object after the change.

In [71]:
def read_obj_file(file_path):
    vertices = []
    faces = []

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                vertex = list(map(float, line.strip().split()[1:]))
                vertices.append(vertex)
            elif line.startswith('f '):
                face = [int(index.split('/')[0])-1 for index in line.strip().split()[1:]]
                faces.append(face)

    return np.array(vertices), np.array(faces)

In [79]:
# The code below is just some hint for you. Feel free to modify anything if you want!
#The Code Below will output the image of the convex object before changing the movement of the light source
import os
import matplotlib.cm as cm
# Read .obj file
obj_file_path = '/tetrahedron.obj'
#obj_file_path = '/icosahedron_modified.obj'
vertices, faces = read_obj_file(obj_file_path)

center = np.mean(vertices, axis=0)
vertices = vertices - center

max_x = np.max(vertices[:, 0])
min_x = np.min(vertices[:, 0])
max_y = np.max(vertices[:, 1])
min_y = np.min(vertices[:, 1])
max_z = np.max(vertices[:, 2])
min_z = np.min(vertices[:, 2])


# Plot vertices
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_axis_off()

ax.set_xlim(min_x, max_x), ax.set_ylim(min_y, max_y), ax.set_zlim(min_z, max_z)
ax.view_init(azim=60, elev=30)

lookat = compute_lookat(ax.azim, ax.elev)
basecolor = np.array([0, 1, 0.7])
phi = 2 * np.pi
lightsource = np.array([np.cos(phi), np.sin(phi), np.sin(phi)])
lightsource = lightsource / np.linalg.norm(lightsource)
num_faces = len(faces)
colormap = cm.get_cmap('hsv', num_faces)
colors = [colormap(i / num_faces) for i in range(num_faces)]
# [TODO] Plot faces
######## write you code below
for i, face in enumerate(faces):
        poly3d = [[vertices[vertice_index] for vertice_index in face]]
        ax.add_collection3d(Poly3DCollection(poly3d, facecolors=colors[i], linewidths=1, edgecolors='blue', alpha=0.5))

#plt.show()
studentID = '112006221'
plt.savefig(f'HW3_{studentID}.png') # replace with your own studentID
plt.close(fig)


<ipython-input-79-b392098c6194>:35: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('hsv', num_faces)


In [81]:
#The Code Below will output the image of the convex object after changing the movement of the light source
import os
import matplotlib.cm as cm
# Read .obj file
obj_file_path = '/tetrahedron.obj'
#obj_file_path = '/icosahedron_modified.obj'
vertices, faces = read_obj_file(obj_file_path)

center = np.mean(vertices, axis=0)
vertices = vertices - center

max_x = np.max(vertices[:, 0])
min_x = np.min(vertices[:, 0])
max_y = np.max(vertices[:, 1])
min_y = np.min(vertices[:, 1])
max_z = np.max(vertices[:, 2])
min_z = np.min(vertices[:, 2])

# Plot vertices
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_axis_off()
ax.set_xlim(min_x, max_x), ax.set_ylim(min_y, max_y), ax.set_zlim(min_z, max_z)
ax.view_init(azim=60, elev=30)

ambient_intensity =  3
light_scale = 6
lookat = compute_lookat(ax.azim, ax.elev)
basecolor = np.array([0, 1, 0.7])
num_frames = 36
light_radius = 4
light_elevation = 1
phi = 2 * np.pi
lightsource = np.array([np.cos(phi), np.sin(phi), np.sin(phi)])
lightsource = lightsource / np.linalg.norm(lightsource)
num_faces = len(faces)
colormap = cm.get_cmap('hsv', num_faces)
colors = [colormap(i / num_faces) for i in range(num_faces)]
for frame in range(num_frames):
    phi = 2 * np.pi * frame / num_frames
    lightsource = np.array([light_radius * np.cos(phi), light_radius * np.sin(phi), light_elevation])

    ax.cla()
    ax.set_axis_off()
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    ax.set_zlim(min_z, max_z)
# [TODO] Plot faces
######## write you code below
    for i, face in enumerate(faces):
      poly3d = [[vertices[vertice_index] for vertice_index in face]]
      v0 = vertices[face[0]]
      v1 = vertices[face[1]]
      v2 = vertices[face[2]]

      normal = np.cross(v1 - v0, v2 - v0)
      normal = normal / np.linalg.norm(normal)
      light_direction = lightsource - np.mean(poly3d[0], axis=0)
      light_direction = light_direction / np.linalg.norm(light_direction)

      intensity = np.dot(normal, light_direction)
      intensity = np.clip(intensity * light_scale + ambient_intensity, 0, 1)

      face_color = colormap(i / num_faces)[:3]
      shaded_color = intensity * np.array(face_color)
      ax.add_collection3d(Poly3DCollection(poly3d, facecolors=shaded_color, linewidths=1, edgecolors='blue', alpha=0.5))
#plt.show()
studentID = '112006221'
plt.savefig(f'HW3_{studentID}.png') # replace with your own studentID
plt.close(fig)


<ipython-input-81-2f08d493ce35>:37: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('hsv', num_faces)


## Problem 6
There is a type of terms, called **vn** (vertex normal), in the obj file.
However, it record vn as each

Please look up the definition of vertex normal and implement it accordingly. The OBJ
file with the corrected vertex normals is icosahedron modified.obj.

In [74]:
from collections import defaultdict

def load_obj(file_path):
    vertices = []
    faces = []
    vertex_normals = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue

            values = line.split()
            if not values:
                continue

            if values[0] == 'v':
                vertices.append([float(x) for x in values[1:4]])
            elif values[0] == 'vn':
                vertex_normals.append([float(x) for x in values[1:4]])
            elif values[0] == 'f':
                vertex_indices = []
                normal_indices = []

                for vertex in values[1:]:
                    # deal with v//vn
                    if '//' in vertex:
                        v_idx, _, n_idx = vertex.split('/')
                        vertex_indices.append(int(v_idx) - 1)
                        normal_indices.append(int(n_idx) - 1)
                    # deal with v
                    else:
                        vertex_indices.append(int(vertex) - 1)

                faces.append(vertex_indices)

    return np.array(vertices), np.array(faces), np.array(vertex_normals)

In [75]:
# [TODO]
def compute_vertex_normals(vertices, faces):

    vertex_normals = np.zeros_like(vertices)
    ######## write you code below
    for face in faces:
        v1_idx, v2_idx, v3_idx = face
        v1, v2, v3 = vertices[v1_idx], vertices[v2_idx], vertices[v3_idx]
        edge1 = v2 - v1
        edge2 = v3 - v1

        face_normal = np.cross(edge1, edge2)
        face_normal = face_normal / np.linalg.norm(face_normal)

        vertex_normals[v1_idx] += face_normal
        vertex_normals[v2_idx] += face_normal
        vertex_normals[v3_idx] += face_normal
    for i in range(len(vertices)):
        if np.linalg.norm(vertex_normals[i]) > 0:
            vertex_normals[i] /= np.linalg.norm(vertex_normals[i])
    #########

    return vertex_normals

In [76]:
obj_path = "/icosahedron_modified.obj"
vertices, faces, vn = load_obj(obj_path)
vertex_normals = compute_vertex_normals(vertices, faces)

#print("vertex normal：")
#for i, normal in enumerate(vertex_normals):
     #print(f"v{i+1:02d} normal: [{normal[0]:.6f}, {normal[1]:.6f}, {normal[2]:.6f}]")
if (np.allclose(vertex_normals, vn)):
    print("PASS")
else :
    print("ERROR")

PASS
